## Set up Tensorflow

In [1]:
import tensorflow as tf
print("TensorFlow version: ", tf.__version__)

2022-11-18 12:47:56.378683: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-18 12:47:56.458366: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-18 12:47:56.461568: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-18 12:47:56.461576: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

TensorFlow version:  2.10.0


## Load Dataset
`MNIST Dataset`

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 250.0, x_test / 250.0

## Build a machine learning model
`tf.keras.Sequential`

In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
])

2022-11-18 12:53:55.586054: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-18 12:53:55.586070: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-18 12:53:55.586082: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (nsl54): /proc/driver/nvidia/version does not exist
2022-11-18 12:53:55.586284: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


#### For each example, the model returns a vector of logits or log-odds scores, one for each class

In [5]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.5271358 , -0.9132867 , -0.9680246 ,  0.31074375, -0.13945785,
         0.24067312, -0.09968281, -0.16331375,  0.00785332, -0.01689196]],
      dtype=float32)

#### The `tf.nn.softmax` function converts these logits to probabilities for each class

In [6]:
tf.nn.softmax(predictions).numpy()

array([[0.17416073, 0.04124608, 0.03904904, 0.14027245, 0.08942363,
        0.13077992, 0.09305214, 0.0873156 , 0.10361645, 0.1010839 ]],
      dtype=float32)

It is possible to bake the `tf.nn.softmax` function into the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output.

#### Define a loss function for training using `losses.SparseCategoricalCrossentropy`, which takes a vector of logits and a True index and returns a scalar loss for each example

In [7]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

In [8]:
loss_fn(y_train[:1], predictions).numpy()

2.0342393

#### Before training configure and compile the model using Keras `Model.compile`. Set the optimizer class to adam, set the loss to the `loss_fn` function you defined earlier, and specify a metric to be evaluated for the model by setting the metrics parameter to accuracy

In [9]:
model.compile(optimizer='adam',
             loss=loss_fn,
             metrics=['accuracy'])

## Train and Evaluate
`model.fit`

In [11]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 1s 720us/step - loss: 0.0403 - accuracy: 0.9867
Epoch 2/5
1875/1875 [==============================] - 1s 698us/step - loss: 0.0383 - accuracy: 0.9870
Epoch 3/5
1875/1875 [==============================] - 1s 696us/step - loss: 0.0351 - accuracy: 0.9878
Epoch 4/5
1875/1875 [==============================] - 1s 693us/step - loss: 0.0336 - accuracy: 0.9887
Epoch 5/5
1875/1875 [==============================] - 1s 685us/step - loss: 0.0334 - accuracy: 0.9888


#### The `Model.evaluate` method checks the models performance, usually on a "Validation-set" or "Test-set"

In [12]:
model.evaluate(x_test, y_test, verbose=2)

313/313 - 0s - loss: 0.0822 - accuracy: 0.9806 - 226ms/epoch - 723us/step


[0.08222439885139465, 0.9805999994277954]

Wrap the trained model, and attach the softmax to it to return probability

In [16]:
probability_model = tf.keras.Sequential([
    model,
    tf.keras.layers.Softmax()
])

In [17]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[8.6248800e-11, 2.9011930e-11, 2.5391597e-10, 5.7377110e-07,
        4.9048398e-19, 2.9150113e-10, 2.1216288e-17, 9.9999928e-01,
        1.8720160e-10, 9.4567596e-08],
       [1.2594133e-10, 7.8273894e-07, 9.9999917e-01, 6.6204402e-09,
        1.2580502e-24, 5.6347626e-12, 1.8060770e-11, 2.1010581e-25,
        2.1842463e-10, 1.6253757e-24],
       [1.6989303e-10, 9.9994028e-01, 1.3656324e-05, 3.4130064e-08,
        1.2639210e-06, 1.4871054e-10, 2.5875855e-07, 4.0226383e-05,
        4.3486689e-06, 2.6736494e-10],
       [9.9998772e-01, 4.2822022e-14, 1.2082480e-05, 7.0695956e-09,
        1.0812663e-13, 2.2377828e-08, 1.4310130e-07, 1.2114230e-09,
        2.0952741e-11, 1.2647242e-09],
       [2.7043157e-08, 2.5702765e-17, 8.2003460e-08, 8.6414547e-12,
        9.9985027e-01, 2.8526091e-11, 3.8063497e-06, 2.4275107e-06,
        6.7926154e-10, 1.4338974e-04]], dtype=float32)>